In [16]:
import numpy as np
import torch

from utils import get_loaders

tweets = np.load("data/tweets.npy")
labels = np.load("data/hate_original.npy")

train_on_gpu = torch.cuda.is_available()
print("Using GPU." if train_on_gpu else "Using CPU.")

Using GPU.


### Creating the Training, Validation, and Test Sets

In [17]:
train_loader, valid_loader, test_loader = get_loaders(tweets, labels)

			Feature Shapes:
Train set: 		(19826, 91) 
Validation set: 	(2478, 91) 
Test set: 		(2479, 91)
